In [51]:
# Imports
import dateutil.parser
import matplotlib.pyplot as plt
import pandas
import sys

In [44]:
# Read raw sentence data
sentence_data = pandas.read_table('/data/workspace/wellsettled-research/scratch/results/ws_match_sentences.txt',
                  sep='|',
                  header=None)
sentence_data.columns = ['nan', 'sentence_id', 'case_caption', 'case_date', 'sentence']
del sentence_data['nan']

# Read sentence match data
sentence_match_data = pandas.read_csv('/data/workspace/wellsettled-research/scratch/results/ws_matches.csv',
                                      index_col=0)
                                      
sentence_match_data.columns = ['sentence_id', 'case_caption', 'case_date', 'phrase_id']

# Read phrase data
phrase_data = pandas.read_table('/data/workspace/wellsettled-research/scratch/results/ws_phrase_mapping.csv',
                                header=None)
phrase_data.columns = ['phrase_id', 'phrase_stem']

In [45]:
sentence_data.head()

,sentence_id,case_caption,case_date,sentence
0,5d8c2fa4-4260-49e1-8ef1-e8d40c9339b2,"PENNELL v. SAN JOSE, 485 U.S. 1 (1988)",1988-02-24,"As appellants point out, """"[i]t is axiomatic t..."
1,02da221e-2b90-4a4e-90fe-ebc010f30b2d,"EDENFIELD v. FANE, 507 U.S. 761 (1993)",1993-04-26,"It is well established that """"[t]he party seek..."
2,26fc841a-e7d3-4377-8b01-2a7dc461cc35,"LEWIS v. CITY OF NEW ORLEANS, 415 U.S. 130 (1974)",1974-02-20,There are certain well-defined and narrowly li...
3,7e7a37cc-01a4-4756-9c82-835c227cda64,"WEEKS v. ANGELONE, 528 U.S. 225 (2000)",2000-01-19,A jury is presumed to follow its instructions....
4,2b436cbc-b29f-42ae-b7a9-4bdffb7d69c1,"BOARD OF TRUSTEES, UNIVERSITY OF ALABAMA v. GA...",2001-02-21,The ultimate guarantee of the Eleventh Amendme...


In [46]:
sentence_match_data.head()

,sentence_id,case_caption,case_date,phrase_id
0,5d8c2fa4-4260-49e1-8ef1-e8d40c9339b2,"PENNELL v. SAN JOSE, 485 U.S. 1 (1988)",1988-02-24,7551
1,02da221e-2b90-4a4e-90fe-ebc010f30b2d,"EDENFIELD v. FANE, 507 U.S. 761 (1993)",1993-04-26,3196
2,26fc841a-e7d3-4377-8b01-2a7dc461cc35,"LEWIS v. CITY OF NEW ORLEANS, 415 U.S. 130 (1974)",1974-02-20,2069
3,7e7a37cc-01a4-4756-9c82-835c227cda64,"WEEKS v. ANGELONE, 528 U.S. 225 (2000)",2000-01-19,6212
4,2b436cbc-b29f-42ae-b7a9-4bdffb7d69c1,"BOARD OF TRUSTEES, UNIVERSITY OF ALABAMA v. GA...",2001-02-21,3692


In [47]:
phrase_data.head()

,phrase_id,phrase_stem
0,0,defend seek revers basi insuffici evid swim up...
1,1,right assist counsel waiv
2,2,must consid state prior art shown refer employ...
3,3,privat properti land extend low-wat mark
4,4,order warrant find neglig proxim caus injuri m...


In [48]:
# Merge all data
merge_data = sentence_match_data.merge(phrase_data, on='phrase_id').merge(sentence_data, on='sentence_id')
merge_data = merge_data[['sentence_id', 'case_caption_x', 'case_date_x', 'phrase_id', 'phrase_stem', 'sentence']]
merge_data.columns = ['sentence_id', 'case_caption', 'case_date', 'phrase_id', 'phrase_stem', 'sentence']
merge_data['case_date'] = merge_data['case_date'].apply(dateutil.parser.parse)

In [49]:
# Now sort/check
merge_data.sort(columns=['phrase_id', 'case_date'], ascending=True, inplace=True)
merge_data.to_csv('merged_wsr_matches.csv')

In [67]:
def check_sentence_match(stems):
    """Check if the sentence stem sequence matches our desired
    patterns."""

    # Check minimum length
    if len(stems) < 3:
        return False

    # Require initial "it"
    if stems[0] != "it":
        return False

    # Check for "well*"
    if not stems[1].startswith(u'well'):
        return False

    # Now handle hyphen vs. space
    if '-' in stems[1]:
        tokens = stems[1].split(u'-')
        if tokens[1] in [u'settl', u'establish']:
            return True
    elif stems[2] in [u'settl', u'establish']:
        return True
    else:
        return False

In [76]:
# Handle imports
sys.path.append('/data/workspace/wellsettled-research/src/')
from wsr.process.stem import process_sentence

# Define nested function for sentence
def is_sentence_ws(sentence):
    return check_sentence_match(process_sentence(sentence.decode("utf-8")))

# Do it live
merge_data['is_ws'] = merge_data['sentence'].apply(is_sentence_ws)

In [85]:
# Get the first well-settled instances
first_ws_instances = merge_data.loc[merge_data['is_ws']==True].groupby('phrase_id').head(1)
first_ws_instances.to_csv('first_ws_instances.csv')
first_ws_id = first_ws_instances.index

# Update original merge DF
merge_data['is_first_ws'] = False
merge_data.loc[first_ws_id, 'is_first_ws'] = True
merge_data.to_csv('merged_wsr_matches.csv')